<a href="https://colab.research.google.com/github/daisukemiyajima/portfolio/blob/main/%E8%A8%80%E8%AA%9E%E3%83%A2%E3%83%87%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sat Oct 15 19:16:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    41W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!pip install transformers datasets sentencepiece
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from pathlib import Path
from multiprocessing import cpu_count
import random
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error, f1_score

from datasets import load_dataset, Dataset, DatasetDict

from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from transformers import EvalPrediction
from transformers import set_seed

import torch
from torch import nn

In [ ]:
!pip install transformers datasets sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df_test = pd.read_csv(INPUT_DIR/"test_dataset.csv")
df_train = pd.read_csv(INPUT_DIR/"train_dataset.csv")
dftemp = pd.concat([df_train,df_test])
bunshoudf = pd.concat([dftemp["dimainmidashi"],dftemp["honbun"],dftemp["thema"],dftemp["image_caption"],dftemp["twitter_label"]],axis =1)

In [ ]:
train = bunshoudf[:len(df_train)]
test = bunshoudf[len(df_train):]

In [ ]:
del test["twitter_label"]

In [ ]:
EXP_NAME = 'exp003'
INPUT_DIR = Path('/content/drive/MyDrive/signate/Input')
OUTPUT_DIR = Path('/content/drive/MyDrive/signate/Output')
DEBUG = False
TEXT_COLUMNS = test.columns
MODEL_NAME = 'microsoft/deberta-v3-large'
N_SPLIT = 2
TRN_FOLDS = [0]
MAX_LEN = 512
SEED = 3090
training_args = TrainingArguments(
    output_dir=str(OUTPUT_DIR/EXP_NAME),
    seed=SEED,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    weight_decay=0.01,
    fp16=True,
    gradient_accumulation_steps=1,
    evaluation_strategy='epoch',
    save_total_limit=1,
    save_strategy ='epoch',    
    metric_for_best_model='f1_score',
    load_best_model_at_end=True,
    greater_is_better=True
)


In [ ]:
def create_folds(data, num_splits):
    data["kfold"] = -1

    mskf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)
    data_labels = data['twitter_label']

    for f, (t_, v_) in enumerate(mskf.split(data, data_labels)):
        data.loc[v_, "kfold"] = f

    return data

data = train
data = create_folds(data, num_splits=N_SPLIT)
data.to_csv("train_folds.csv", index=False)
print("Folds created successfully")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Folds created successfully


In [ ]:
pd.read_csv("train_folds.csv")

,dimainmidashi,honbun,thema,image_caption,twitter_label,kfold
0,「理想主義者と言われても平和な世界目指す」千葉の15歳核廃絶訴え,今年の千葉県原爆死没者慰霊式典で、県立安房高校1年で館山市の武士俣（ぶしまた）拓真さん（1...,ＷＰ戦後７７年｜核兵器廃絶を訴える高校生,武士俣拓真さん（右）と古畑玲子さん=2022年7月22日午前11時47分、館山市、伊藤繭莉撮影,0.0,1
1,自決前、大田中将が海軍次官にあてた電文（全文）,76年前の1945年6月、沖縄の地下に掘られた洞穴で、一人の軍人が自ら命を絶ちました。海軍...,連載｜遺された戦後～大田家 それぞれの道,沖縄の海軍司令官だった大田実氏（中央）一家の家族写真=板垣愛子さん提供,1.0,1
2,成人式、開催か延期か 分かれる対応,成人の日（11日）を前にした10日、備前市などで成人式が開かれた。この日に予定していた岡山...,成人式,備前市の成人式に参加し、マスク姿で記念撮影する新成人たち=2021年1月10日午前9時41分...,0.0,1
3,ファスト映画を投稿の疑い、ユーチューバーを逮捕 容疑を否認,映画を無断で短く編集した「ファスト映画」をネットに投稿したとして、宮城県警は15日、神奈川...,ファスト映画,逮捕された男が投稿していたファスト映画。冒頭には「ファストシネマ」と掲げ、自画像とみられるイ...,1.0,1
4,【写真まとめ】鳥取城北×東海大相模の熱戦を振り返る,2年ぶりに球春到来――。第93回選抜高校野球大会が3月19日、阪神甲子園球場で開幕しました...,２０２１センバツ写真まとめ,力投する鳥取城北の山内=柴田悠貴撮影,0.0,0
...,...,...,...,...,...,...
118090,性的暴行後に絞殺か、男を再逮捕へ 豊島区の女性遺棄,東京都豊島区の会社員女性（当時35）が昨年9月に行方不明になり、12月に栃木県那須町の土中...,殺人,警視庁本部,1.0,0
118091,こま蛇が弁財天を守る 花の季節の川口・興禅院,埼玉県川口市安行領家の興禅院境内にある弁財天を守る「こま蛇」が人気を集めている。この地域に...,川口・興禅院,愛らしいこま蛇=2021年4月7日、埼玉県川口市安行領家の興禅院,0.0,0
118092,プーチン大統領、ザポリージャ原発を国有化 大統領令に署名,ロシアのプーチン大統領は5日、ロシアが一方的に併合したウクライナ中南部ザポリージャ州にある...,ウクライナ（６日夕刊）＝Ｄ山根,ウクライナ中南部のザポリージャ原発で2022年9月22日、国際原子力機関（IAEA）の専門家...,0.0,0
118093,中低所得ほど苦しい…スクリューフレーション深刻 生活必需品が高騰,総務省が22日発表した6月の消費者物価指数（生鮮食品を除く）は、3カ月連続で前年同月比2%...,朝刊,野菜売り場で買い物する人たち=2022年5月、東京都練馬区,1.0,1


In [ ]:
def text_to_input_ids(examples):
    return tokenizer(examples['text'], padding=False, truncation=True, max_length=MAX_LEN)

def connect_text(df, text_cols, sep):
    text_df = df[text_cols].fillna('NAN').astype(str)
    connected_text = text_df[text_cols[0]].str.cat(text_df[text_cols[1:]], sep=sep)

    return connected_text

In [ ]:
class RandomMask:
    def __init__(self, tokenizer, prob=0.5, mask_prob=0.15):
        self.mask_token_id = tokenizer.mask_token_id
        self.prob = prob
        self.mask_prob = mask_prob

    def __call__(self, examples):        
        if random.random() > self.prob:
            examples['input_ids'] = [self._mask(input_ids) for input_ids in examples['input_ids']]

        return examples

    def _mask(self, input_ids: list) -> list:
        length = len(input_ids)
        mask_idx = random.sample(range(1, length), int(length*self.mask_prob)) # random masking except [CLS]
        for i in mask_idx:
            input_ids[i] = self.mask_token_id

        return input_ids

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get('labels').float()
        _inputs = {k: v for k, v in inputs.items() if k != 'labels'} # to not automatically calculate losses
        
        outputs = model(**_inputs)
        logits = outputs.get('logits')
        
        loss_fn = nn.BCEWithLogitsLoss()
        loss = loss_fn(logits.view(-1), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

def threshold_search(y_true, y_proba):
    best_threshold = 0
    best_score = 0
    for threshold in [i * 0.01 for i in range(100)]:
        score = f1_score(y_true=y_true, y_pred=y_proba > threshold)
        if score > best_score:
            best_threshold = threshold
            best_score = score
    search_result = {'threshold': best_threshold, 'f1': best_score}
    return search_result

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def compute_metrics(p: EvalPrediction):
    preds = sigmoid(p.predictions)
    labels = p.label_ids
    score = threshold_search(labels, preds)['f1']
    metrics = {'f1_score': score}    
    
    return metrics

In [ ]:
if __name__ == "__main__":
    df = pd.read_csv('train_folds.csv')
    test_df = test
    if DEBUG:
        df = df.head(50)
        test_df = test_df.head(50)
        training_args.num_train_epochs = 1

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # for dynamic paddnig
    train_transform = RandomMask(tokenizer, prob=0.5, mask_prob=0.1) # [MASK] augmentation

    df['text'] = connect_text(df, TEXT_COLUMNS, tokenizer.sep_token)
    df['label'] = df['twitter_label']
    ds = Dataset.from_pandas(df[['text', 'label', 'kfold']])
    ds = ds.map(text_to_input_ids, batched=True, num_proc=cpu_count())

    test_df['text'] = connect_text(test_df, TEXT_COLUMNS, tokenizer.sep_token)
    test_ds = Dataset.from_pandas(test_df[['text']])
    test_ds = test_ds.map(text_to_input_ids, batched=True, num_proc=cpu_count())
    test_ds = test_ds.remove_columns(['text'])

    oof = []
    labels = []
    test_preds = []
    for fold in TRN_FOLDS:
        set_seed(SEED)
        ds_i = DatasetDict(
            {
                'train': ds.filter(lambda x: x['kfold']!=fold),
                'eval': ds.filter(lambda x: x['kfold']==fold),
            }
        )
        ds_i = ds_i.remove_columns(['text', 'kfold'])
        ds_i['train'].set_transform(train_transform)

        # train
        model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=1)
        training_args.output_dir = str((OUTPUT_DIR / EXP_NAME) / f'fold_{fold}')
        trainer = CustomTrainer(
            model=model,
            args=training_args,
            train_dataset=ds_i["train"],
            eval_dataset=ds_i["eval"],
            tokenizer=tokenizer,
            data_collator=data_collator,
            compute_metrics=compute_metrics,
        )
        trainer.train()

        # inference
        oof_i = trainer.predict(ds_i['eval'])
        test_preds_i = trainer.predict(test_ds).predictions
        oof.append(sigmoid(oof_i.predictions))
        labels.append(oof_i.label_ids)
        test_preds.append(test_preds_i)
        pd.DataFrame(oof).to_csv("oof.csv",index = False)

    oof = np.vstack(oof).flatten()
    labels = np.hstack(labels)
    search_result = threshold_search(labels, oof)
    print('OOF Score: ', search_result['f1'], 'Threshold:', search_result['threshold'])


    a = (np.mean(test_preds, axis=0) > search_result['threshold']).astype(int)

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/580 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#0:   0%|          | 0/10 [00:00<?, ?ba/s]

...

Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.classifier.weight', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

Epoch,Training Loss,Validation Loss,F1 Score
1,0.523000,0.502986,0.762039


***** Running Evaluation *****
  Num examples = 59048
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/signate/Output/exp003/fold_0/checkpoint-3691
Configuration saved in /content/drive/MyDrive/signate/Output/exp003/fold_0/checkpoint-3691/config.json
Model weights saved in /content/drive/MyDrive/signate/Output/exp003/fold_0/checkpoint-3691/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/signate/Output/exp003/fold_0/checkpoint-3691/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/signate/Output/exp003/fold_0/checkpoint-3691/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:746: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  scale, dtype=query_layer.dtype
/usr/local/lib/python3.7/dist-packages/transformers/

Epoch,Training Loss,Validation Loss,F1 Score
1,0.523000,0.502986,0.762039
2,0.471800,0.517059,0.772237
3,0.411900,0.473589,0.779800


Saving model checkpoint to /content/drive/MyDrive/signate/Output/exp003/fold_0/checkpoint-7382
Configuration saved in /content/drive/MyDrive/signate/Output/exp003/fold_0/checkpoint-7382/config.json
Model weights saved in /content/drive/MyDrive/signate/Output/exp003/fold_0/checkpoint-7382/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/signate/Output/exp003/fold_0/checkpoint-7382/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/signate/Output/exp003/fold_0/checkpoint-7382/special_tokens_map.json
Deleting older checkpoint [/content/drive/MyDrive/signate/Output/exp003/fold_0/checkpoint-3691] due to args.save_total_limit
/usr/local/lib/python3.7/dist-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:746: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  scale, dtype=query_layer.dtype

In [ ]:
pd.DataFrame(sigmoid(test_preds[0])).to_csv((OUTPUT_DIR/EXP_NAME)/"test_stacking.csv",index = False)

In [ ]:
pd.DataFrame(oof).to_csv((OUTPUT_DIR/EXP_NAME)/"oof_stacking.csv",index = False)

In [ ]:
pd.DataFrame(labels).to_csv((OUTPUT_DIR/EXP_NAME)/"labels_stacking.csv",index = False)

In [ ]:
data = pd.read_csv("train_folds.csv")